In [19]:
# !pip show langchain==0.1.11

from dotenv import load_dotenv
load_dotenv("/home/haolan/.env")

from IPython.display import display
from IPython.display import Markdown
import textwrap


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

from langchain_google_genai import ChatGoogleGenerativeAI
import os
GOOGLE_API_KEY=os.environ.get("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-pro",
                               google_api_key=GOOGLE_API_KEY,
                               convert_system_message_to_human=True)

In [29]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent


#tools = load_tools(["ddg-search"], llm=llm)


In [32]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

@tool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a+b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [41]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = [search, multiply, add]

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

print(agent.agent.llm_chain.prompt)


input_variables=['agent_scratchpad', 'input'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nsearch: search(query: str) -> str - Look up things online., args: {{\'query\': {{\'title\': \'Query\', \'type\': \'string\'}}}}\nmultiply: multiply(a: int, b: int) -> int - Multiply two numbers., args: {{\'a\': {{\'title\': \'A\', \'type\': \'integer\'}}, \'b\': {{\'title\': \'B\', \'type\': \'integer\'}}}}\nadd: add(a: int, b: int) -> int - Add two numbers., args: {{\'a\': {{\'title\': \'A\', \'type\': \'integer\'}}, \'b\': {{\'title\': \'B\', \'type\': \'integer\'}}}}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or search, multiply, add\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_inpu

In [43]:
agent.run("What is the result of 12376*3456 + 2?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "multiply",
  "action_input": {
    "a": 12376,
    "b": 3456
  }
}
```
Observation: 42771456
Thought:And I need to add 2 to that.
Action:
```
{
  "action": "add",
  "action_input": {
    "a": 42771456,
    "b": 2
  }
}
```
Observation: 42771458
Thought:That's the final answer.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The final answer is 42771458."
}
```

> Finished chain.


'The final answer is 42771458.'

In [44]:
# multi-input issue: https://github.com/langchain-ai/langchain/issues/3700
# Structured tools: https://blog.langchain.dev/structured-tools/